<a href="https://colab.research.google.com/github/TejParmar10/stock_market_prediction/blob/main/stock_market_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
url='https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv'
data= pd.read_csv(
    url,
    sep=',',
    encoding='utf-8',
)
print(data)
data.isna().any()

            Date    Open    High     Low    Last   Close  \
0     2018-09-28  234.05  235.95  230.20  233.50  233.75   
1     2018-09-27  234.55  236.80  231.10  233.80  233.25   
2     2018-09-26  240.00  240.00  232.50  235.00  234.25   
3     2018-09-25  233.30  236.75  232.00  236.25  236.10   
4     2018-09-24  233.55  239.20  230.75  234.00  233.30   
...          ...     ...     ...     ...     ...     ...   
2030  2010-07-27  117.60  119.50  112.00  118.80  118.65   
2031  2010-07-26  120.10  121.00  117.10  117.10  117.60   
2032  2010-07-23  121.80  121.95  120.25  120.35  120.65   
2033  2010-07-22  120.30  122.00  120.25  120.75  120.90   
2034  2010-07-21  122.10  123.00  121.05  121.10  121.55   

      Total Trade Quantity  Turnover (Lacs)  
0                  3069914          7162.35  
1                  5082859         11859.95  
2                  2240909          5248.60  
3                  2349368          5503.90  
4                  3423509          7999.55  
...

Date                    False
Open                    False
High                    False
Low                     False
Last                    False
Close                   False
Total Trade Quantity    False
Turnover (Lacs)         False
dtype: bool

In [2]:
df=pdr.get_data_tiingo('AAPL',api_key='b8ff655b05592100b3894dcd2b7efcc2ad825a74')

<ipython-input-2-1ed07f8ac249>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df=pdr.get_data_tiingo('AAPL',api_key='b8ff655b05592100b3894dcd2b7efcc2ad825a74')


In [3]:
!pip install pandas_datareader
import pandas_datareader as pdr

api_key = 'b8ff655b05592100b3894dcd2b7efcc2ad825a74'

In [4]:
df.to_csv('AAPL.csv')

In [5]:
import pandas as pd
df=pd.read_csv('AAPL.csv')
df.head()

,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,AAPL,2019-02-11 00:00:00+00:00,169.43,171.21,169.25,171.05,20993425,40.776245,41.204633,40.732925,41.166126,83973700,0.0,1.0
1,AAPL,2019-02-12 00:00:00+00:00,170.89,171.00,169.70,170.10,22283523,41.127619,41.154092,40.841225,40.937492,89134092,0.0,1.0
2,AAPL,2019-02-13 00:00:00+00:00,170.18,172.48,169.92,171.39,22490233,40.956745,41.510280,40.894172,41.247953,89960932,0.0,1.0
3,AAPL,2019-02-14 00:00:00+00:00,170.80,171.26,169.38,169.71,21835747,41.105959,41.216666,40.764212,40.843632,87342988,0.0,1.0
4,AAPL,2019-02-15 00:00:00+00:00,170.42,171.70,169.75,171.25,24626814,41.014505,41.322559,40.853258,41.214259,98507256,0.0,1.0


In [6]:
df1=df.reset_index()['close']

In [7]:
df1.shape

(1259,)

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))


In [9]:
training_size=int(len(df1)*0.65)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

In [10]:
def create_dataset(dataset,time_step=1):
  dataX,dataY=[],[]
  for i in range(len(dataset)-time_step-1):
    a=dataset[i:(i+time_step),0]
    dataX.append(a)
    dataY.append(dataset[i+ time_step,0])
  return np.array(dataX),np.array(dataY)

In [11]:
time_step=100
X_train,Y_train=create_dataset(train_data,time_step)
X_test,Y_test=create_dataset(test_data,time_step)

In [12]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [14]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 lstm_1 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50851 (198.64 KB)
Trainable params: 50851 (198.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,batch_size=64,verbose=1)

Epoch 1/100
12/12 [==============================] - 9s 290ms/step - loss: 0.0424 - val_loss: 0.0012
Epoch 2/100
12/12 [==============================] - 3s 248ms/step - loss: 0.0119 - val_loss: 0.0011
Epoch 3/100
12/12 [==============================] - 2s 183ms/step - loss: 0.0103 - val_loss: 3.5812e-04
Epoch 4/100
12/12 [==============================] - 2s 167ms/step - loss: 0.0088 - val_loss: 2.8218e-04
Epoch 5/100
12/12 [==============================] - 2s 164ms/step - loss: 0.0077 - val_loss: 0.0013
Epoch 6/100
12/12 [==============================] - 2s 163ms/step - loss: 0.0083 - val_loss: 3.5195e-04
Epoch 7/100
12/12 [==============================] - 2s 164ms/step - loss: 0.0069 - val_loss: 6.2847e-04
Epoch 8/100
12/12 [==============================] - 3s 240ms/step - loss: 0.0068 - val_loss: 0.0020
Epoch 9/100
12/12 [==============================] - 2s 189ms/step - loss: 0.0057 - val_loss: 2.5066e-04
Epoch 10/100
12/12 [==============================] - 3s 257ms/step - l

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(Y_train,train_predict))

In [ ]:
math.sqrt(mean_squared_error(Y_test,test_predict))

In [ ]:
import matplotlib.pyplot as plt
look_back=100
trainPredictplot=np.empty_like(df1)
trainPredictplot[:, :]=np.nan
trainPredictplot[look_back:len(train_predict)+look_back,:]=train_predict
testPredictplot=np.empty_like(df1)
testPredictplot[: , :]=np.nan
testPredictplot[len(train_predict)+(look_back*2)+1:len(df1)-1,:]=test_predict
plt.plot(scaler.inverse_transform(df1))
plt.plot(trainPredictplot)
plt.plot(testPredictplot)
plt.show()

In [ ]:
x_input=test_data[341:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
lst_output=[]
n_steps=100
i=0
while(i<30):
  if(len(temp_input)>100):
    x_input=np.array(temp_input[1:])
    print("{} day input {}".format(i,x_input))
    x_input=x_input.reshape(1,-1)
    x_input=x_input.reshape((1,n_steps,1))
    yhat=model.predict(x_input,verbose=0)
    print("{} day output {}".format(i,yhat))
    temp_input.extend(yhat[0].tolist())
    temp_input=temp_input[1:]
    lst_output.extend(yhat.tolist())
    i=i+1
  else:
    x_input=x_input.reshape((1,n_steps,1))
    yhat=model.predict(x_input,verbose=0)
    print(yhat[0])
    temp_input.extend(yhat[0].tolist())
    print(len(temp_input))
    lst_output.extend(yhat.tolist())
    i=i+1
print(lst_output)


In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,131)

In [ ]:
df3=df1.tolist()
df3.extend(lst_output)

In [ ]:
plt.plot(day_new,scale.inverse_transform(df1))